## Consigna

Toda la información necesaria para realizar este trabajo la encontraremos en este link:  
https://www.wienerborse.at/en/market-data/shares-others/historical-data/?ISIN=ES0178430E18&ID_NOTATION=199457683&cHash=d6f32e62c796e401c618cfbef65e3d27&c48840%5BDATETIME_TZ_START_RANGE%5D=09%2F01%2F2021&c48840%5BDATETIME_TZ_END_RANGE%5D=08%2F31%2F2022

(Lo cambie del que esta en la consigna https://es.finance.yahoo.com/quote/TEF/history?p=TEF&.tsrc=fin-srch porque no me funcionaba y me arrojaba un 404)

La empresa Yellow necesita obtener los valores relevantes sobre los históricos en la bolsa de valores de la competencia, de esa manera pueden estimar el estado actual de la compañía respecto a sus competidores.

El objetivo de esta actividad es obtener el resultado de las siguientes métricas: (Ajuste un poco porque me parece que estaba mal pedir el maximo de apertura en todas)

1. Valores de apertura
- Día y valor en el cual la apertura es el máximo
- Día y valor en el cual el valor es el mínimo

2. Valores de cierre
- Día y valor en el cual el cierre es el máximo
- Día y valor en el cual el valor es el mínimo

3. Valores de volumen
- Día y valor en el cual el volumen es el máximo
- Día y valor en el cual el valor es el mínimo

In [1]:
# Importo las librerías necesarias
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
# Se selecciona la URL con la información
url = "https://www.wienerborse.at/en/market-data/shares-others/historical-data/?ISIN=ES0178430E18&ID_NOTATION=199457683&cHash=d6f32e62c796e401c618cfbef65e3d27&c48840%5BDATETIME_TZ_START_RANGE%5D=09%2F01%2F2021&c48840%5BDATETIME_TZ_END_RANGE%5D=08%2F31%2F2022"
htlm = urlopen(url)

In [3]:
# Con esto se crea un objeto de la data en la URL
soup = BeautifulSoup(htlm, 'lxml')
type(soup)

bs4.BeautifulSoup

In [4]:
# Se imprimen y guardan en un df las filas de la tabla
rows = soup.find_all('tr')
print(rows[:5])

[<tr><th class="large-font" data-tablesaw-priority="persist">Last</th><th class="large-font">Chg. %</th><th>Chg. Abs.</th><th>ISIN</th><th>Date, Time</th><th class="kv-align-right mpq">MPQ</th></tr>, <tr data-key="0"><td class="multi-cell large-font"><span class="large-font"><i class="indicator indicator-negative"></i></span><span class="large-font">3.6570</span></td><td class="large-font"><span class="negative">-1.14%</span></td><td><span class="negative">-0.0420</span></td><td>ES0178430E18</td><td>09/23/2022, 17:32:34</td><td class="multi-cell mpq kv-align-right"><span class="mpq status-tooltip-link mpqD" data-title="Closed">D</span></td></tr>, <tr><th data-tablesaw-priority="persist">Date</th><th class="kv-align-right">Open</th><th class="kv-align-right">High</th><th class="kv-align-right">Low</th><th class="kv-align-right">Last Close</th><th class="kv-align-right">Chg.%</th><th class="kv-align-right">Total Value<sup>1</sup></th><th class="kv-align-right">Total Volume<sup>1</sup></t

In [5]:
# Se toma los valores de la tabla y se agrupan en un dataframe
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

[<td>11/10/2021</td>, <td class="kv-align-right">3.8295</td>, <td class="kv-align-right">3.9345</td>, <td class="kv-align-right">3.8295</td>, <td class="kv-align-right">3.9345</td>, <td class="kv-align-right"><span class="positive">+3.01%</span></td>, <td class="kv-align-right">57,009</td>, <td class="kv-align-right">14,736</td>]


bs4.element.ResultSet

In [6]:
# Extraer los datos sin los tags de HTLM
import re

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

[11/10/2021, 3.8295, 3.9345, 3.8295, 3.9345, +3.01%, 57,009, 14,736]


str

In [7]:
# Convertir la lista en un dataframe
df = pd.DataFrame(list_rows)
df.head(10)

,0
0,[]
1,"[3.6570, -1.14%, -0.0420, ES0178430E18, 09/23/..."
2,[]
3,"[09/01/2021, 4.2250, 4.2330, 4.2250, 4.2295, +..."
4,"[09/02/2021, 4.2385, 4.2385, 4.2050, 4.2050, -..."
5,"[09/03/2021, 4.1970, 4.1970, 4.1315, 4.1315, -..."
6,"[09/06/2021, 4.1685, 4.1685, 4.1450, 4.1475, +..."
7,"[09/07/2021, 4.2005, 4.2030, 4.2005, 4.2030, +..."
8,"[09/08/2021, 4.2110, 4.2255, 4.2110, 4.2255, +..."
9,"[09/09/2021, 4.2335, 4.2335, 4.1505, 4.1750, -..."


In [8]:
# Ahora hay que ajustar los datos en una tabla que podamos manipular
df1 = df[0].str.split(',', expand=True)
df1[0] = df1[0].str.strip('[')
col_labels = soup.find_all('th')
df1.head(10)

,0,1,2,3,4,5,6,7,8,9
0,],None,None,None,None,None,None,None,None,None
1,3.6570,-1.14%,-0.0420,ES0178430E18,09/23/2022,17:32:34,D],None,None,None
2,],None,None,None,None,None,None,None,None,None
3,09/01/2021,4.2250,4.2330,4.2250,4.2295,+1.27%,5,926,1,400]
4,09/02/2021,4.2385,4.2385,4.2050,4.2050,-0.58%,0,0],None,None
5,09/03/2021,4.1970,4.1970,4.1315,4.1315,-1.75%,12,489,3,000]
6,09/06/2021,4.1685,4.1685,4.1450,4.1475,+0.39%,16,580,4,000]
7,09/07/2021,4.2005,4.2030,4.2005,4.2030,+1.34%,0,0],None,None
8,09/08/2021,4.2110,4.2255,4.2110,4.2255,+0.54%,0,0],None,None
9,09/09/2021,4.2335,4.2335,4.1505,4.1750,-1.20%,14,311,3,448]


In [9]:
# Se eliminan los valores nulos
df_drop = df1.dropna(axis=0)
df_drop.head(10)

,0,1,2,3,4,5,6,7,8,9
3,09/01/2021,4.2250,4.2330,4.2250,4.2295,+1.27%,5,926,1,400]
5,09/03/2021,4.1970,4.1970,4.1315,4.1315,-1.75%,12,489,3,000]
6,09/06/2021,4.1685,4.1685,4.1450,4.1475,+0.39%,16,580,4,000]
9,09/09/2021,4.2335,4.2335,4.1505,4.1750,-1.20%,14,311,3,448]
10,09/10/2021,4.2295,4.2295,4.1315,4.1315,-1.04%,25,353,6,092]
11,09/13/2021,4.1240,4.1735,4.1240,4.1735,+1.02%,41,318,9,966]
12,09/14/2021,4.1745,4.1795,4.1435,4.1435,-0.72%,13,575,3,248]
13,09/15/2021,4.1390,4.1490,4.1365,4.1365,-0.17%,21,127,5,092]
16,09/20/2021,4.1140,4.1140,4.1070,4.1100,-1.26%,22,805,5,552]
17,09/21/2021,4.0905,4.0940,4.0520,4.0620,-1.17%,69,666,17,108]


In [10]:
# Como hace falta tener los nombres de las columnas y los tags de Beautiful Soup toman data de una tabla de arriba, se crea un dataframe con los nombres de las colñumans para luego concatenar

col_labels = ["Date,Open,High,Low,Close,Chg.%,Total Value1,Total Value2,Total Volume1,Total Volume2"]
df2 = pd.DataFrame(col_labels)
df2.head()

,0
0,"Date,Open,High,Low,Close,Chg.%,Total Value1,To..."


In [11]:
df3 = df2[0].str.split(',', expand=True)
df_drop.head()

,0,1,2,3,4,5,6,7,8,9
3,09/01/2021,4.2250,4.2330,4.2250,4.2295,+1.27%,5,926,1,400]
5,09/03/2021,4.1970,4.1970,4.1315,4.1315,-1.75%,12,489,3,000]
6,09/06/2021,4.1685,4.1685,4.1450,4.1475,+0.39%,16,580,4,000]
9,09/09/2021,4.2335,4.2335,4.1505,4.1750,-1.20%,14,311,3,448]
10,09/10/2021,4.2295,4.2295,4.1315,4.1315,-1.04%,25,353,6,092]


In [12]:
# Se concatenan los dos dataframes para 

frames = [df3, df_drop]
df4 = pd.concat(frames)
df4.head(5)

,0,1,2,3,4,5,6,7,8,9
0,Date,Open,High,Low,Close,Chg.%,Total Value1,Total Value2,Total Volume1,Total Volume2
3,09/01/2021,4.2250,4.2330,4.2250,4.2295,+1.27%,5,926,1,400]
5,09/03/2021,4.1970,4.1970,4.1315,4.1315,-1.75%,12,489,3,000]
6,09/06/2021,4.1685,4.1685,4.1450,4.1475,+0.39%,16,580,4,000]
9,09/09/2021,4.2335,4.2335,4.1505,4.1750,-1.20%,14,311,3,448]


In [13]:
# Ahora se asignan los nombres de la primera fila como headers

df5 = df4.rename(columns=df4.iloc[0])
df6 = df5.drop(df5.index[0])
df6.head(5)

,Date,Open,High,Low,Close,Chg.%,Total Value1,Total Value2,Total Volume1,Total Volume2
3,09/01/2021,4.2250,4.2330,4.2250,4.2295,+1.27%,5,926,1,400]
5,09/03/2021,4.1970,4.1970,4.1315,4.1315,-1.75%,12,489,3,000]
6,09/06/2021,4.1685,4.1685,4.1450,4.1475,+0.39%,16,580,4,000]
9,09/09/2021,4.2335,4.2335,4.1505,4.1750,-1.20%,14,311,3,448]
10,09/10/2021,4.2295,4.2295,4.1315,4.1315,-1.04%,25,353,6,092]


In [14]:
# Como las columnas de value se separaron por el split inical las volví a unir agregando el separador de la coma (,)
df6["Total_Value"] = df6["Total Value1"] + "," + df6["Total Value2"]
df6["Total_Volume"] = df6["Total Volume1"] + "," + df6["Total Volume2"]
df6.head(5)

,Date,Open,High,Low,Close,Chg.%,Total Value1,Total Value2,Total Volume1,Total Volume2,Total_Value,Total_Volume
3,09/01/2021,4.2250,4.2330,4.2250,4.2295,+1.27%,5,926,1,400],"5,926","1,400]"
5,09/03/2021,4.1970,4.1970,4.1315,4.1315,-1.75%,12,489,3,000],"12,489","3,000]"
6,09/06/2021,4.1685,4.1685,4.1450,4.1475,+0.39%,16,580,4,000],"16,580","4,000]"
9,09/09/2021,4.2335,4.2335,4.1505,4.1750,-1.20%,14,311,3,448],"14,311","3,448]"
10,09/10/2021,4.2295,4.2295,4.1315,4.1315,-1.04%,25,353,6,092],"25,353","6,092]"


In [15]:
df6.info()
df6.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 3 to 52
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date           40 non-null     object
 1   Open           40 non-null     object
 2   High           40 non-null     object
 3   Low            40 non-null     object
 4   Close          40 non-null     object
 5   Chg.%          40 non-null     object
 6   Total Value1   40 non-null     object
 7   Total Value2   40 non-null     object
 8   Total Volume1  40 non-null     object
 9   Total Volume2  40 non-null     object
 10  Total_Value    40 non-null     object
 11  Total_Volume   40 non-null     object
dtypes: object(12)
memory usage: 4.1+ KB


(40, 12)

Hasta acá lo que se realizó fue el limpiado de la base de datos para poder analizar y empezar a contestar la información. Ahora es tiempo de contestar las preguntas: 

1. Valores de apertura
- Día y valor en el cual la apertura es el máximo
- Día y valor en el cual el valor es el mínimo

2. Valores de cierre
- Día y valor en el cual el cierre es el máximo
- Día y valor en el cual el valor es el mínimo

3. Valores de volumen
- Día y valor en el cual el volumen es el máximo
- Día y valor en el cual el valor es el mínimo

In [16]:
print(df6[df6.Open == df6.Open.max()])

         Date     Open     High      Low    Close    Chg.% Total Value1  \
9  09/09/2021   4.2335   4.2335   4.1505   4.1750   -1.20%           14   

  Total Value2 Total Volume1 Total Volume2 Total_Value Total_Volume  
9          311             3          448]      14,311       3,448]  


In [17]:
print(df6[df6.Open == df6.Open.min()])

          Date     Open     High      Low    Close    Chg.% Total Value1  \
49  11/05/2021   3.6810   3.7750   3.6770   3.7750   +3.13%          100   

   Total Value2 Total Volume1 Total Volume2 Total_Value Total_Volume  
49          909            27          290]     100,909      27,290]  


- Máximo de apertura: El 09/09/2021 y el valor fue 4.2335
- Mínimo de apertura: El 11/05/2021 y el valor fue 3.6810

In [18]:
print(df6[df6.Close == df6.Close.max()])

         Date     Open     High      Low    Close    Chg.% Total Value1  \
3  09/01/2021   4.2250   4.2330   4.2250   4.2295   +1.27%            5   

  Total Value2 Total Volume1 Total Volume2 Total_Value Total_Volume  
3          926             1          400]       5,926       1,400]  


In [19]:
print(df6[df6.Close == df6.Close.min()])

          Date     Open     High      Low    Close    Chg.% Total Value1  \
48  11/04/2021   3.6970   3.7305   3.6255   3.6605   -2.46%          147   

   Total Value2 Total Volume1 Total Volume2 Total_Value Total_Volume  
48          167            40          090]     147,167      40,090]  


- Máximo de cierre: El 09/01/2021 y el valor fue 4.2295
- Mínimo de cierre: El 11/04/2021 y el valor fue 3.6605

In [25]:
print(df6[df6.Total_Volume == df6.Total_Volume.max()])

          Date     Open     High      Low    Close    Chg.% Total Value1  \
11  09/13/2021   4.1240   4.1735   4.1240   4.1735   +1.02%           41   

   Total Value2 Total Volume1 Total Volume2 Total_Value Total_Volume  
11          318             9          966]      41,318       9,966]  


In [26]:
print(df6[df6.Total_Volume == df6.Total_Volume.min()])

         Date     Open     High      Low    Close    Chg.% Total Value1  \
3  09/01/2021   4.2250   4.2330   4.2250   4.2295   +1.27%            5   

  Total Value2 Total Volume1 Total Volume2 Total_Value Total_Volume  
3          926             1          400]       5,926       1,400]  


- Máximo de volumen: El 09/13/2021 y el valor fue 9,966
- Mínimo de volumen: El 09/01/2021  y el valor fue 1,400